In [7]:
import pandas as pd

df = pd.read_csv('../separate_dfs/deposit_accumulative_data.csv')
df['date'] = pd.to_datetime(df['date'])

# Fill balance data
df['avg_monthly_balance_KZT'] = df.groupby('client_code')['avg_monthly_balance_KZT'].transform('first')

# 1. Balance features
df['balance_median'] = df.groupby('client_code')['avg_monthly_balance_KZT'].transform('median')
df['balance_cv'] = df.groupby('client_code')['avg_monthly_balance_KZT'].transform(lambda x: x.std() / x.mean() if x.mean() > 0 else 0)

# 2. Deposit features
deposit_types = ['salary_in', 'stipend_in', 'family_in', 'deposit_topup_out', 'cashback_in', 'refund_in']
df['is_deposit'] = df['type'].isin(deposit_types).astype(int)
df['deposit_count'] = df.groupby('client_code')['is_deposit'].transform('sum')

# Calculate deposit regularity at client level (not row level)
deposit_dates = df[df['is_deposit']==1].groupby('client_code')['date'].apply(lambda x: x.diff().dt.days.std()).fillna(0)
df['deposit_regularity'] = df['client_code'].map(deposit_dates).fillna(0)

# 3. Inflow features
df['is_inflow'] = ((df['data_source'] == 'transfer') & (df['direction'] == 'in')).astype(int)
df['inflow_count'] = df.groupby('client_code')['is_inflow'].transform('sum')

# Calculate inflow sums and averages at client level
inflow_sums = df[df['is_inflow']==1].groupby('client_code')['amount'].sum().fillna(0)
inflow_means = df[df['is_inflow']==1].groupby('client_code')['amount'].mean().fillna(0)
df['inflow_amount_sum'] = df['client_code'].map(inflow_sums).fillna(0)
df['avg_inflow_amount'] = df['client_code'].map(inflow_means).fillna(0)

# 4. Outflow features
df['is_outflow'] = ((df['data_source'] == 'transaction') | ((df['data_source'] == 'transfer') & (df['direction'] == 'out'))).astype(int)
df['outflow_count'] = df.groupby('client_code')['is_outflow'].transform('sum')

outflow_sums = df[df['is_outflow']==1].groupby('client_code')['amount'].sum().fillna(0)
df['outflow_amount_sum'] = df['client_code'].map(outflow_sums).fillna(0)
df['accumulation_ratio'] = df['inflow_amount_sum'] / (df['outflow_amount_sum'] + 1)
df['net_flow'] = df['inflow_amount_sum'] - df['outflow_amount_sum']

# 5. Spending features
spending_categories = ['Продукты питания', 'Кафе и рестораны', 'Одежда и обувь', 'Спорт', 'Книги', 'Косметика и Парфюмерия']
df['is_target_spending'] = df['category'].isin(spending_categories).astype(int)
df['spending_frequency'] = df.groupby('client_code')['is_target_spending'].transform('sum')

spending_medians = df[df['is_target_spending']==1].groupby('client_code')['amount'].median().fillna(0)
spending_stds = df[df['is_target_spending']==1].groupby('client_code')['amount'].std().fillna(0)
df['spending_amount_median'] = df['client_code'].map(spending_medians).fillna(0)
df['spending_amount_std'] = df['client_code'].map(spending_stds).fillna(0)
df['spending_consistency'] = 1 / (1 + df['spending_amount_std'] / (df['spending_amount_median'] + 1))

print(f"Created {len(df.columns)} features")
print(f"Sample features: {df.columns.tolist()[:10]}")
df

Created 32 features
Sample features: ['client_code', 'name', 'status', 'age', 'city', 'avg_monthly_balance_KZT', 'date', 'data_source', 'type', 'category']


,client_code,name,status,age,city,avg_monthly_balance_KZT,date,data_source,type,category,...,is_outflow,outflow_count,outflow_amount_sum,accumulation_ratio,net_flow,is_target_spending,spending_frequency,spending_amount_median,spending_amount_std,spending_consistency
0,1,Айгерим,Зарплатный клиент,NaN,Алматы,92643.0,2025-06-02 12:10:21,transaction,NaN,Продукты питания,...,1,112,1164024.65,1.610867,711065.99,1,112,8937.45,6000.022167,0.598351
1,1,Айгерим,Зарплатный клиент,NaN,Алматы,92643.0,2025-06-04 12:10:35,transaction,NaN,Продукты питания,...,1,112,1164024.65,1.610867,711065.99,1,112,8937.45,6000.022167,0.598351
2,1,Айгерим,Зарплатный клиент,NaN,Алматы,92643.0,2025-06-04 17:00:58,transaction,NaN,Кафе и рестораны,...,1,112,1164024.65,1.610867,711065.99,1,112,8937.45,6000.022167,0.598351
3,1,Айгерим,Зарплатный клиент,NaN,Алматы,92643.0,2025-06-04 17:40:05,transaction,NaN,Кафе и рестораны,...,1,112,1164024.65,1.610867,711065.99,1,112,8937.45,6000.022167,0.598351
4,1,Айгерим,Зарплатный клиент,NaN,Алматы,92643.0,2025-06-04 19:00:08,transaction,NaN,Кафе и рестораны,...,1,112,1164024.65,1.610867,711065.99,1,112,8937.45,6000.022167,0.598351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,60,Ермек,Зарплатный клиент,NaN,Кызылорда,48779.0,2025-08-31 10:00:00,transaction,NaN,Кафе и рестораны,...,1,123,1262660.64,1.151359,191116.65,1,123,8862.35,5894.889440,0.600570
6376,60,Ермек,Зарплатный клиент,NaN,Кызылорда,48779.0,2025-08-31 11:20:39,transaction,NaN,Кафе и рестораны,...,1,123,1262660.64,1.151359,191116.65,1,123,8862.35,5894.889440,0.600570
6377,60,Ермек,Зарплатный клиент,NaN,Кызылорда,48779.0,2025-08-31 12:10:15,transaction,NaN,Продукты питания,...,1,123,1262660.64,1.151359,191116.65,1,123,8862.35,5894.889440,0.600570
6378,60,Ермек,Зарплатный клиент,NaN,Кызылорда,48779.0,2025-08-31 17:00:58,transaction,NaN,Кафе и рестораны,...,1,123,1262660.64,1.151359,191116.65,1,123,8862.35,5894.889440,0.600570


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from scipy.stats import percentileofscore
from sklearn.cluster import KMeans



class DataDrivenScoringFramework:
    def __init__(self, product_name):
        self.product_name = product_name
        self.feature_importance = {}
        self.client_features = None
        self.scores = None
    
    def aggregate_to_client_level(self, df):
        """Aggregate with only columns that exist"""
        
        # Define aggregation only for columns that exist
        available_cols = df.columns.tolist()
        
        agg_dict = {}
        
        # Map of desired columns and their aggregation
        desired_aggs = {
            'avg_monthly_balance_KZT': 'first',
            'balance_median': 'first',
            'balance_cv': 'first',
            'deposit_count': 'first',
            'deposit_regularity': 'first',
            'inflow_count': 'first',
            'inflow_amount_sum': 'first',
            'avg_inflow_amount': 'first',
            'outflow_count': 'first',
            'outflow_amount_sum': 'first',
            'accumulation_ratio': 'first',
            'net_flow': 'first',
            'spending_frequency': 'first',
            'spending_amount_median': 'first',
            'spending_amount_std': 'first',
            'spending_consistency': 'first'
        }
        
        # Only include columns that exist
        for col, agg_func in desired_aggs.items():
            if col in available_cols:
                agg_dict[col] = agg_func
        
        client_df = df.groupby('client_code').agg(agg_dict).reset_index()
        
        # Add derived features
        if 'inflow_count' in client_df.columns and 'outflow_count' in client_df.columns:
            client_df['inflow_regularity'] = client_df['inflow_count'] / (client_df['outflow_count'] + 1)
        
        if 'net_flow' in client_df.columns and 'inflow_amount_sum' in client_df.columns:
            client_df['savings_rate'] = client_df['net_flow'] / (client_df['inflow_amount_sum'] + 1)
        
        if 'spending_frequency' in client_df.columns and 'outflow_count' in client_df.columns:
            client_df['transaction_diversity'] = client_df['spending_frequency'] / (client_df['outflow_count'] + 1)
        
        return client_df
    
    def calculate_feature_importance(self, client_df):
        """Calculate feature importance using multiple methods"""
        
        feature_cols = [col for col in client_df.columns if col != 'client_code']
        
        # Prepare data
        X = client_df[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
        X_scaled = StandardScaler().fit_transform(X)
        
        importance_scores = {}
        
        # Method 1: PCA-based importance
        n_components = min(5, len(feature_cols), len(client_df) - 1)
        if n_components > 0:
            pca = PCA(n_components=n_components)
            pca.fit(X_scaled)
            
            # Get loadings for components that explain most variance
            n_important_components = min(3, n_components)
            loadings = np.abs(pca.components_[:n_important_components, :]).mean(axis=0)
            pca_importance = dict(zip(feature_cols, loadings / (loadings.sum() + 1e-10)))
        else:
            pca_importance = {col: 1/len(feature_cols) for col in feature_cols}
        
        # Method 2: Variance-based importance
        variances = X.var()
        if variances.sum() > 0:
            variance_scores = 1 - np.abs(variances - variances.median()) / (variances.max() + 1e-10)
            variance_importance = dict(zip(feature_cols, variance_scores / (variance_scores.sum() + 1e-10)))
        else:
            variance_importance = {col: 1/len(feature_cols) for col in feature_cols}
        
        # Method 3: Correlation uniqueness
        if len(client_df) > 1:
            corr_matrix = X.corr().abs()
            np.fill_diagonal(corr_matrix.values, 0)
            uniqueness = 1 - corr_matrix.mean(axis=0)
            uniqueness_importance = dict(zip(feature_cols, uniqueness / (uniqueness.sum() + 1e-10)))
        else:
            uniqueness_importance = {col: 1/len(feature_cols) for col in feature_cols}
        
        # Combine methods (skip anomaly detection for stability)
        for feature in feature_cols:
            importance_scores[feature] = (
                pca_importance.get(feature, 0) * 0.40 +
                variance_importance.get(feature, 0) * 0.30 +
                uniqueness_importance.get(feature, 0) * 0.30
            )
        
        # Normalize
        total = sum(importance_scores.values())
        if total > 0:
            self.feature_importance = {k: v/total for k, v in importance_scores.items()}
        else:
            self.feature_importance = {k: 1/len(feature_cols) for k in feature_cols}
        
        return self.feature_importance
    
    def calculate_client_scores(self, client_df):
        """Calculate scores using data-driven weights"""
        
        feature_cols = [col for col in client_df.columns if col != 'client_code']
        
        # Normalize features to 0-1
        for col in feature_cols:
            values = client_df[col].fillna(0).replace([np.inf, -np.inf], 0)
            if len(set(values)) > 1:  # Only normalize if there's variance
                client_df[f'{col}_norm'] = [percentileofscore(values, x)/100 for x in values]
            else:
                client_df[f'{col}_norm'] = 0.5  # Default to middle if no variance
        
        # Calculate weighted score
        client_df['composite_score'] = 0
        for feature in feature_cols:
            if f'{feature}_norm' in client_df.columns:
                weight = self.feature_importance.get(feature, 0)
                client_df['composite_score'] += client_df[f'{feature}_norm'] * weight
        
        # Ensure 0-1 range
        client_df['composite_score'] = client_df['composite_score'].clip(0, 1)
        
        # Add percentile rank
        client_df['score_percentile'] = client_df['composite_score'].rank(pct=True)
        
        # Recommendation tiers
        client_df['recommendation_tier'] = pd.cut(
            client_df['composite_score'],
            bins=[0, 0.3, 0.5, 0.7, 1.0],
            labels=['Low', 'Medium', 'High', 'Very High']
        )
        
        return client_df
    
    def fit_score(self, df):
        """End-to-end scoring pipeline"""
        
        # Step 1: Aggregate
        print(f"\nAggregating {len(df)} rows to client level...")
        client_df = self.aggregate_to_client_level(df)
        print(f"Created {len(client_df)} client profiles")
        
        # Step 2: Calculate feature importance
        print("Calculating data-driven feature importance...")
        self.calculate_feature_importance(client_df)
        
        print("\nTop 5 Most Important Features:")
        top_features = sorted(self.feature_importance.items(), key=lambda x: x[1], reverse=True)[:5]
        for feat, importance in top_features:
            print(f"  {feat}: {importance:.3f}")
        
        # Step 3: Calculate scores
        print("\nCalculating client scores...")
        client_df = self.calculate_client_scores(client_df)
        
        self.client_features = client_df
        self.scores = client_df[['client_code', 'composite_score', 'score_percentile', 'recommendation_tier']]
        
        return self.scores

# Apply the framework
scorer = DataDrivenScoringFramework('deposit_accumulative')
scores = scorer.fit_score(df)

print("\nScore Distribution:")
print(scores['recommendation_tier'].value_counts())
print("\nTop 10 Clients:")
print(scores.nlargest(10, 'composite_score'))


Aggregating 6380 rows to client level...
Created 44 client profiles
Calculating data-driven feature importance...

Top 5 Most Important Features:
  avg_inflow_amount: 0.067
  inflow_count: 0.066
  deposit_count: 0.062
  inflow_amount_sum: 0.061
  savings_rate: 0.060

Calculating client scores...

Score Distribution:
recommendation_tier
High         24
Medium       19
Very High     1
Low           0
Name: count, dtype: int64

Top 10 Clients:
    client_code  composite_score  score_percentile recommendation_tier
6            12         0.715894          1.000000           Very High
38           49         0.689498          0.977273                High
4             8         0.641200          0.954545                High
5            10         0.624162          0.931818                High
28           37         0.614453          0.909091                High
23           30         0.611974          0.886364                High
0             1         0.600694          0.863636       